# Training and Evaluation

We will take a first-pass at evaluating or technique to start understanding its efficacy. We will existing CNN architectures and evaluate its performance on our interested categories with and without using our interested categories.

In [1]:
import cv2
import datetime
from matplotlib import pyplot as plt
import numpy as np
import sys
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

np.random.seed(123)

In [2]:
# NOTE: Copied from clustering NB
def load_metadata(filename):
    with open(filename, 'r') as f:
        return [x.strip().split('\t') for x in f.readlines()]

In [3]:
INPUT_SHAPE=(64,64,3)

def get_mobilenet(input_shape=INPUT_SHAPE):
    application = tf.keras.applications.MobileNet(input_shape=input_shape, include_top=False)
    for i in range(len(application.layers)):
        application.layers[i].trainable = i < 10
    return tf.keras.Sequential([
        application,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(200, activation='softmax'),
    ])

In [4]:
def get_simplecnn(input_shape=INPUT_SHAPE):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(512, (3,3), (1,1), input_shape=input_shape, activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(512, (2,2), (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(256, (2,2), (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(256, (2,2), (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(200),
    ])

In [5]:
# simplecnn = get_simplecnn()
# simplecnn.summary()

In [6]:
mobilenet = get_mobilenet()
mobilenet.summary()

/home/dennis/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 2, 2, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               1

In [7]:
@tf.function
def decode_img(image):
    img = tf.image.decode_jpeg(image, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img, [64, 64])

@tf.function
def load_image_data(path, label):
    img_data = tf.io.read_file(path)
    img = decode_img(img_data)
    return img, label

In [8]:
def load_labels(metadata):
    labels = np.array([x[1] for x in metadata])
    distinct_labels = np.array([[x] for x in set(labels)])
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(distinct_labels)
    y_train = encoder.transform([[x] for x in labels])
#     y_train = np.array([x[1] for x in np.argwhere(y_train == 1)])
    return (y_train, encoder)

In [9]:
# TODO: Remove hardcoding
print('Loading data into memory...')
train_metadata = load_metadata('./metadata_output/train_metadata.txt')
(y_train, encoder) = load_labels(train_metadata)

# Interested indices for test data filtering
interested_categories = ['n01882714', 'n04562935']
interested_one_hot = encoder.transform([[x] for x in interested_categories])
interested_indices = np.array([x[1] for x in np.argwhere(interested_one_hot == 1)])
print('Done.')

Loading data into memory...
Done.


In [10]:
# Encoding sanity checks;
# assert(len(train_metadata) == len(y_train))
# assert(len(set(y_train)) == 200)
assert(np.count_nonzero(y_train == 1) == len(train_metadata))
# print(y_train)

In [23]:
# Split data into train and validation sets
paths_and_labels = [(train_metadata[x][0], y_train[x]) for x in range(len(y_train))]
np.random.shuffle(paths_and_labels)
num_valid = int(len(paths_and_labels) * 0.1)
valid_path_and_labels = paths_and_labels[:num_valid]
train_path_and_labels = paths_and_labels[num_valid:]

# Convert into a TF dataset
list_ds = tf.data.Dataset.from_generator(
    lambda: train_path_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
list_ds = list_ds.map(lambda x,y: load_image_data(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

list_ds = list_ds.cache()
list_ds = list_ds.repeat()
list_ds = list_ds.batch(30)
list_ds = list_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valid_ds = tf.data.Dataset.from_generator(
    lambda: valid_path_and_labels,
    (tf.string, tf.int32),
    (tf.TensorShape([]), tf.TensorShape([len(y_train[0])]))
)
valid_ds = valid_ds.map(lambda x,y: load_image_data(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.cache()
valid_ds = valid_ds.repeat()
valid_ds = valid_ds.batch(30)
valid_ds = valid_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [24]:
# TODO: Shouldn't be hardcoded.
TRAIN_STEPS=3000
TEST_STEPS=500
NUM_EPOCHS=250
TEST_TRAIN_SPLIT=0.33
def train_model(model, dataset, valid_dataset, name):    
    # Compile model                                                                                                      
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-3),                                                           
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),                                  
                  metrics=['accuracy'])      
    
    # Stop early if we're not making good progress                                                                           
    early_stop_monitor = tf.keras.callbacks.EarlyStopping(                                                                   
        monitor='val_loss',                                                                                              
        restore_best_weights=True,                                                                                       
        patience=10)   

    # Prepare for checkpoints            
    checkpoint_path = './checkpoints/' + name + '/cp-{epoch:04d}.ckpt'                                   
    cp_callback = tf.keras.callbacks.ModelCheckpoint(                                                                    
        filepath=checkpoint_path,                                                                                    
        verbose=1,                                                                                                   
        save_weights_only=False,                                                                                     
        save_freq=25000000)

    # Tensorboard                                                                                                        
    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")                                              
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    history = model.fit(                                                                                                 
        x=dataset,
        epochs=NUM_EPOCHS,                                                                                                  
        steps_per_epoch=TRAIN_STEPS,
        callbacks=[tensorboard_callback, cp_callback, early_stop_monitor],
        use_multiprocessing=True,
        validation_steps=num_valid,
        validation_data=valid_dataset,
        shuffle=True)

    return history

In [13]:
# Evaluate model on interesting inputs
def evaluate_model(model, test_sets):
    for test_set in test_sets:
        X = test_set[0]
        y = test_set[1]
        model.evaluate(X, y)

In [25]:
# Train and save model
train_model(mobilenet, list_ds, valid_ds, 'mobilenet_imbalanced')
mobilenet.save("/models/{}".format('mobilenet_imbalanced'))
print('model saved')

Train for 1000 steps, validate for 14000 steps
Epoch 1/250
1000/1000 [==============================] - 226s 226ms/step - loss: 5.2323 - accuracy: 0.0767 - val_loss: 5.2300 - val_accuracy: 0.0786
Epoch 2/250
1000/1000 [==============================] - 117s 117ms/step - loss: 5.2244 - accuracy: 0.0836


KeyboardInterrupt: 

In [ ]:
# TODO: Load test data, filter for interest, evaluate model